In [1]:
import os, math, subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# some settings for displaying Pandas results
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', -1)

In [35]:
ls home-credit-default-risk


 Volume in drive C has no label.
 Volume Serial Number is 6C43-4112

 Directory of C:\Users\PC\Documents\home-credit-default-risk

01/17/2020  10:46 PM    <DIR>          .
01/17/2020  10:46 PM    <DIR>          ..
12/11/2019  02:58 AM        26,567,651 application_test.csv
12/11/2019  02:59 AM       166,133,370 application_train.csv
12/11/2019  02:59 AM       170,016,717 bureau.csv
12/11/2019  02:59 AM       375,592,889 bureau_balance.csv
12/11/2019  03:00 AM       424,582,605 credit_card_balance.csv
12/11/2019  02:58 AM            37,383 HomeCredit_columns_description.csv
12/11/2019  03:00 AM       723,118,349 installments_payments.csv
12/11/2019  02:58 AM       392,703,158 POS_CASH_balance.csv
12/11/2019  03:02 AM       404,973,293 previous_application.csv
12/11/2019  03:02 AM           536,202 sample_submission.csv
              10 File(s)  2,684,261,617 bytes
               2 Dir(s)  115,555,549,184 bytes free


In [48]:
ls_files = subprocess.check_output(["ls"," home-credit-default-risk"]).decode("utf-8").splitlines()[7:]
ls_files = [f.split()[-1] for f in ls_files]
ls_files = sorted([f for f in ls_files if f not in ["HomeCredit_columns_description.csv"]])
ls_files

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [42]:
def exploring_stats(pdf_input):
    # check rows, cols
    total_records = pdf_input.shape[0]
    total_columns = pdf_input.shape[1]
    print("Total records:", total_records)
    print("Total columns:", total_columns)

    # check dtypes
    name = []
    sub_type = []
    for n, t in pdf_input.dtypes.iteritems():
        name.append(n)
        sub_type.append(t)

    # check distinct
    ls_ndist = []
    for cname in pdf_input.columns:
        ndist = pdf_input[cname].nunique()
        pct_dist = ndist * 100.0 / total_records
        ls_ndist.append("{} ({:0.2f}%)".format(ndist, pct_dist))

    # check missing
    ls_nmiss = []
    for cname in pdf_input.columns:
        nmiss = pdf_input[cname].isnull().sum()
        pct_miss = nmiss * 100.0 / total_records
        ls_nmiss.append("{} ({:0.2f}%)".format(nmiss, pct_miss))

    # check zeros
    ls_zeros = []
    for cname in pdf_input.columns:
        try:
            nzeros = (pdf_input[cname] == 0).sum()
            pct_zeros = nzeros * 100.0 / total_records
            ls_zeros.append("{} ({:0.2f}%)".format(nzeros, pct_zeros))
        except:
            ls_zeros.append("{} ({:0.2f}%)".format(0, 0))
            continue

    # check negative
    ls_neg = []
    for cname in pdf_input.columns:
        try:
            nneg = (pdf_input[cname].astype("float") < 0).sum()
            pct_neg = nneg * 100.0 / total_records
            ls_neg.append("{} ({:0.2f}%)".format(nneg, pct_neg))
        except:
            ls_neg.append("{} ({:0.2f}%)".format(0, 0))
            continue

    # prepare output
    data = {
        "name": name,
        "sub_type": sub_type,
        "n_distinct": ls_ndist,
        "n_miss": ls_nmiss,
        "n_zeros": ls_zeros,
        "n_negative": ls_neg,
    }

    # check stats
    pdf_stats = pdf_input.describe().transpose()
    ls_stats = []
    for stat in pdf_stats.columns:
        data[stat] = []
        for cname in pdf_input.columns:
            try:
                data[stat].append(pdf_stats.loc[cname, stat])
            except:
                data[stat].append(0.0)

    # take samples
    nsample = 10
    pdf_sample = pdf_input.sample(frac=.5).head(nsample).transpose()
    pdf_sample.columns = ["sample_{}".format(i) for i in range(nsample)]

    # output
    col_ordered = ["sub_type", "n_distinct", "n_miss", "n_negative", "n_zeros",
                   "25%", "50%", "75%", "count", "max", "mean", "min", "std"] + list(pdf_sample.columns)
    pdf_data = pd.DataFrame(data).set_index("name")
    pdf_data = pd.concat([pdf_data, pdf_sample], axis=1)
    pdf_data = pdf_data[col_ordered]

    return pdf_data

In [56]:
%%time
# ls_report = {}
# for f in ls_files:
#     print("Exploring {}".format(f))
#     data_path = os.path.join("home-credit-default-risk", f)
#     pdf_data = pd.read_csv(data_path)
#     ls_report[f] = exploring_stats(pdf_data)
#     display(ls_report[f])
print("Exploring: {}".format("csv"))
data_path = os.path.join("home-credit-default-risk","application_test.csv")
pdf_data = pd.read_csv(data_path)
ls       = exploring_stats(pdf_data)
display(ls)

Exploring: csv
Total records: 48744
Total columns: 121


,sub_type,n_distinct,n_miss,n_negative,n_zeros,25%,50%,75%,count,max,mean,min,std,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
name,,,,,,,,,,,,,,,,,,,,,,,
SK_ID_CURR,int64,48744 (100.00%),0 (0.00%),0 (0.00%),0 (0.00%),188557.7500,277549.0000,367555.5000,48744.0,4.5625e+05,2.7780e+05,1.0000e+05,103169.5473,434328,237815,122621,358735,215157,144446,205957,174153,235066,447522
NAME_CONTRACT_TYPE,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans
CODE_GENDER,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,F,F,F,F,F,M,M,F,F,F
FLAG_OWN_CAR,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,N,N,N,N,N,N,Y,N,Y,N
FLAG_OWN_REALTY,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,Y,Y,Y,Y,N,Y,Y,N,N,Y
CNT_CHILDREN,int64,11 (0.02%),0 (0.00%),0 (0.00%),34685 (71.16%),0.0000,0.0000,1.0000,48744.0,2.0000e+01,3.9705e-01,0.0000e+00,0.7090,0,0,0,0,0,1,1,0,0,0
AMT_INCOME_TOTAL,float64,606 (1.24%),0 (0.00%),0 (0.00%),0 (0.00%),112500.0000,157500.0000,225000.0000,48744.0,4.4100e+06,1.7843e+05,2.6942e+04,101522.5915,4.5e+05,1.575e+05,2.025e+05,1.485e+05,1.89e+05,1.44e+05,1.665e+05,1.575e+05,7.65e+04,9e+04
AMT_CREDIT,float64,2937 (6.03%),0 (0.00%),0 (0.00%),0 (0.00%),260640.0000,450000.0000,675000.0000,48744.0,2.2455e+06,5.1674e+05,4.5000e+04,365397.0042,7.57e+05,6.015e+05,3.6e+05,2.606e+05,1.161e+06,2.085e+05,5.51e+05,4.5e+05,1.35e+05,5.715e+05
AMT_ANNUITY,float64,7491 (15.37%),24 (0.05%),0 (0.00%),0 (0.00%),17973.0000,26199.0000,37390.5000,48720.0,1.8058e+05,2.9426e+04,2.2950e+03,16016.3683,4.24e+04,3.084e+04,2.419e+04,2.09e+04,4.185e+04,2.488e+04,4.366e+04,1.907e+04,6890,2.53e+04


Wall time: 4.02 s


In [61]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# some settings for displaying Pandas results
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', -1)
# load train data profile
data_path = "home-credit-default-risk/application_test.csv"
pdf_data = pd.read_csv(data_path)
pdf_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.0188,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.0830,0.2629,0.1394,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.0035,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.3113,0.6222,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.0100,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.5559,0.7296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,